# Trabajo Integrador

## 1. Selección del dataset
Elegimos el **dataset 1**, que contiene datos sobre distintas estaciones meteorológicas de Australia.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [ ]:
# Seteo esta opcion en True para evitar que me exprese los valores numericos con notacion cientifica
np.set_printoptions(suppress=True)

In [ ]:
australian_weather_data = pd.read_csv('./data/weatherAUS.csv',encoding='utf-8',sep=',',skipinitialspace=True)

In [ ]:
australian_weather_data

## 2. Análisis exploratorio inicial

- Visualizar las primeras filas.
- Realizar un resumen de 5 números.
- Identificar los tipos de datos: categórico, ordinal, etc. Responder para cada variable su tipo y si es informativa para un problema de clasificación (por ejemplo si se trata de un código, como una matrícula, o un nombre propio).
- Identificar las variables de entrada y de salida del problema.
- Variables de entrada:
    - Realizar los siguientes análisis por tipo de variable:
        - Numéricas: Obtener conclusiones acerca de la distribución de los datos.
        - Categóricas: Obtener conclusiones acerca de la cardinalidad, representación de cada categoría, etc.
        - Compuestas: ¿Pueden tratarse para utilizarse en el problema a resolver?
- Variables de salida (en caso de aplicar):
    - ¿Están balanceadas las clases?
    - (en caso de aplicar) ¿Qué técnicas consideraría para codificar la variable de salida? Justifique.

**Visualizamos las primeras filas**

In [ ]:
australian_weather_data.head(5)

**Identificamos los tipos de datos**

In [ ]:
australian_weather_data.info()

In [ ]:
australian_weather_data.describe()

| Variable | Descripción | Tipo | Es informativa | Entrada / Salida | 
| -------- | ---------- | ---- | -------------- | ---------------- |
| Date     | Fecha de la medición | Fecha/Hora | Si | Entrada |
| Location | Ubicación geográfica | Categorica Nominal | Si | Entrada |
| MinTemp | Temperatura mínima registrada (celsius) | Numérica Continua | Si | Entrada |
| MaxTemp | Temperatura máxima registrada (celsius) |Numérica Continua | Si | Entrada |
| Rainfall | Precipitaciones registradas (mm) | Numérica Continua | Si | Entrada |
| Evaporation | Evaporación registrada (mm) | Numérica Continua | Si | Entrada |
| Sunshine | Cantidad de horas de sol | Numérica Continua | Si | Entrada |
| WindGustDir | Dirección del viento más fuerte registrado durante el día | Categórica Nominal | Si | Entrada |
| WindGustSpeed | Velocidad del viento más fuerte registrada durante el día (km/h) | Numérica continua | Si | Entrada |
| WindDir9am | Dirección del viento registrada a las 9 am | Categórica Nominal | Si | Entrada |
| WindDir3pm | Dirección del viento registrada a las 3 pm | Categórica Nominal | Si | Entrada |
| WindSpeed9am | Velocidad del viento registrada a las 9 am (km/h) | Numérica continua | Si | Entrada |
| WindSpeed3pm | Velocidad del viento registrada a las 3 pm (km/h) | Numérica continua | Si | Entrada |
| Humidity9am | Humedad registrada a las 9 am (%) | Numérica continua | Si | Entrada |
| Humidity3pm | Humedad registrada a las 3 pm (%) | Numérica continua | Si | Entrada |
| Pressure9am | Presión atmosférica registrada a las 9 am (hpa) | Numérica continua | Si | Entrada |
| Pressure3pm | Presión atmosférica registrada a las 3 pm (hpa) | Numérica continua | Si | Entrada |
| Cloud9am | Fracción del cielo oscurecido por las nubesa las 9 am | Numérica continua | Si | Entrada |
| Cloud3pm | Fracción del cielo oscurecido por las nubesa las 3 pm | Numérica continua | Si | Entrada |
| Temp9am | Temperatura registrada a las 9 am (celsius) | Numérica continua | Si | Entrada |
| Temp3pm | Temperatura registrada a las 3 pm (celsius) | Numérica continua | Si | Entrada |
| RainToday | Flag que indica si llovio durante el día (Yes/No) | Categorica Nominal | Si | Entrada |
| RainTomorrow | Flag que indica si lloverá al día siguiente (Yes/No) | Categórica Nominal | - | Salida |

**Analicemos las features numéricas**

In [ ]:
fig = australian_weather_data.hist(xlabelsize=12, ylabelsize=12,figsize=(22,10))
[x.title.set_size(14) for x in fig.ravel()]
plt.tight_layout()
plt.show()

In [ ]:
def plot_features_distribution(df,cols_to_plot):
  _nrows = len(cols_to_plot)
  _ncols = len(cols_to_plot[0])
  fig, axes = plt.subplots(nrows=_nrows, ncols=_ncols, figsize=(14, 6))
  for x, rows in enumerate(cols_to_plot):
    for y, column in enumerate(rows):
      df[column].plot.density(color='green', ax=axes[x,y])
      axes[x,y].set_title(column.title())

Viendo el histograma de las features numéricas, podemos inferir que:
- Las features **(Rainfall, Evaporation, Cloud9am, Cloud3pm)** tienen una distribución que **no** es normal.
- Las features restantes tienen una distribución normal pero no simétrica, con diferentes oblicuidades.

Proponemos entonces:
- Aplicar transformaciones a las features para normalizarlas.
- Estandarizarlas para escalarlas y que aquellas de mayor magnitud no dominen a las de menor magnitud.

**Analicemos las features categóricas**

Vamos a ver la cardinalidad de las variables categóricas

In [ ]:
australian_weather_data.Location.unique()

In [ ]:
australian_weather_data.WindGustDir.unique()

In [ ]:
australian_weather_data.WindDir9am.unique()

In [ ]:
australian_weather_data.WindDir3pm.unique()

In [ ]:
australian_weather_data.RainToday.unique()

In [ ]:
# Resumimos
cat_features = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

for col in cat_features:
    print('Feature: ', col, 'Number of categories: ', australian_weather_data[col].nunique())

In [ ]:
rows = len(australian_weather_data)

fig,axes = plt.subplots(len(cat_features),1,figsize=(18,len(cat_features)*4))
for i,col in enumerate(cat_features):
    frequencies = pd.Series(australian_weather_data[col].value_counts() / rows)
    frequencies.sort_values(ascending=False).plot.bar(ax=axes[i])
    axes[i].set_xlabel(col)
    axes[i].axhline(y=0.05, color='red')
    axes[i].set_ylabel('Percentage of observations')
    axes[i].set_xlabel('Category')
plt.show()

Vemos que los datos están bastante balanceados. Tanto las features de ubicación geográfica como de dirección del viento presentan una distribución muy parecida a una uniforme.

**Analicemos las variables compuestas**

In [ ]:
australian_weather_data.Date.nunique()

In [ ]:
australian_weather_data.groupby(['Date'])['Date'].count()

Tenemos más de 3000 valores distintos para la variable compuesta fecha/hora. Una opción es transformar esta variable **mapeándola con la estación del año (Otoño, Invierno, Primavera, Verano)**.

**Analicemos el (des)balance de la variable target**

In [ ]:
australian_weather_data.RainTomorrow.unique()

In [ ]:
australian_weather_data_reduced = australian_weather_data.iloc[:,1:]
correlation_matrix = australian_weather_data_reduced.corr(method = 'spearman').round(2)
fig,axes = plt.subplots(1,1,figsize=(20,8))
sns.heatmap(data=correlation_matrix, annot=True,ax=axes)

# ---- #
# Empieza lo de Juani #

**Analicemos los valores nulos**

In [ ]:
# Hago una copia de los datos con nombre más corto para escribir menos
df = australian_weather_data.copy()

# Porcentaje de valores nulos por columna
{df.columns[i]:round(sum(df.iloc[:,i].isnull())/len(df)*100,2) for i in range(len(df.columns))}

In [ ]:
# Primero intento imputar valores nulos de RainTomorrow con el valor de RainToday si es posible.
# Se trata de valores nulos del tipo Missing Not At Random
# Se probó bajar los nulos de RainToday con Rainfall y no redujo el porcentaje de nulos.

df['Date'] = pd.to_datetime(df['Date'])
df['RainTomorrow'].fillna(df['RainToday'].shift(1), inplace=True)

In [ ]:
# El porcentaje de nulos en RainTomorrow bajó a menos del 1%.

{df.columns[i]:round(sum(df.iloc[:,i].isnull())/len(df)*100,2) for i in range(len(df.columns))}

In [ ]:
# Observemos la relación entre los datos faltantes con el equipamiento de la estación metereológica

# Porcentaje de nulos para cada variable en cada estación.
aux = list(df.columns)
aux.remove('Location')
# Primera mitad de las variables
porcentaje_nulos = df.groupby('Location')[aux].apply(lambda x: round(x.isnull().sum() / len(x),4) * 100).reset_index()
porcentaje_nulos.iloc[:,:int(len(aux)/2)]

In [ ]:
# Variables restantes
df.groupby('Location')[aux[int(len(aux)/2):]].apply(lambda x: round(x.isnull().sum() / len(x),4) * 100).reset_index()

Hay varias estaciones con valores faltantes para variables completas. Esto se explica por la ausencia de equipamiento de medición en condiciones operativas.

A modo de ejemplo, la ausencia de valores de evaporación seguramente corresponde a la falta de un tanque clase A:
https://www.hyquestsolutions.com.au/fileadmin/user_upload/Class_A_Evaporation_Pan_en.pdf

In [ ]:
# Se prueba eliminar los lugares con equipamiento faltante y evaluar nuevamente el porcentaje de nulos

porcentaje_nulos[~(porcentaje_nulos == 100).any(axis=1)].reset_index()

In [ ]:
# A modo orientativo se calcula un promedio. Formalmente se deberían pesar los promedios de nulos, pero la
# cantidad de observaciones son similares para cada estación y el número valor obtenido solo es orientativo.

porcentaje_nulos[~(porcentaje_nulos == 100).any(axis=1)].drop(['Location', 'Date'], axis=1).mean()

Tras eliminar (momentáneamente) las locaciones con equipamiento faltante, se observa que las variables 'Evaporation', 'Sunshine', 'Cloud9am' y 'Cloud3pm' aún mantienen una elevada proporción de nulos. Por ello se propone eliminarlas completamente. En algunas locaciones también se observa la falta de equipamiento para medir presión y viento, pero dado el bajo porcentaje de nulos totales que tienen esas variables se propone imputar los valores faltantes según una metodología acorde.

In [ ]:
# Elimino las variables mencionadas

df.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis=1, inplace=True)
df.columns

In [ ]:
# los lugares y valores faltantes son

porcentaje_nulos = porcentaje_nulos.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis=1)
porcentaje_nulos[(porcentaje_nulos == 100).any(axis=1)][['Location','WindGustDir', 'WindGustSpeed', 'Pressure9am', 'Pressure3pm']]

In [ ]:
# Recordemos el porcentaje de nulos totales

{df.columns[i]:round(sum(df.iloc[:,i].isnull())/len(df)*100,2) for i in range(len(df.columns))}

En las estaciones con los valores de viento y presión faltante, el faltante es del tipo *Missing Not At Random* (MNAR). En las restantes estaciones esas mismas variables tienen nulos completamente aleatorios *Missing Completely At Random* (MCAR). Se tratará cada caso en particular.

Entre las variables a imputar valores faltantes, se encuentran las variables categóricas referentes al viento. Debido a que los datos faltantes son aproximadamente el 7%, se propone imputar los valores faltantes con una nueva categoría "FALTANTE" sin que eso genere una categoría poco frecuente.

Para la variable de intensidad máxima de viento, el porcentaje de faltantes se encuentra en el entorno del 7%, valor mayor al 5% límite recomendado para la imputación por media o moda. Se propone tratar la totalidad de los valores nulos como *Missing At Random* (MAR), crear una columna indicando valor faltante e imputar con la mediana debido a que las distribución tienen una marcada oblicuidad.

Para las variables de presión, el porcentaje total de nulos es de aproximadamente un 10% y la cantidad de nulos no aleatorios se encuentra en el entorno del 1%. Se propone agregar una columna indicando los datos faltante y tratar la totalidad de los valores nulos como MAR e imputar con la mediana debido a que las distribuciones tienen una marcada oblicuidad.

In [ ]:
# Reemplazo los valores categóricos mencionados
df['WindGustDir'].fillna('FALTANTE', inplace=True)
df['WindDir9am'].fillna('FALTANTE', inplace=True)
df['WindDir3pm'].fillna('FALTANTE', inplace=True)
{df.columns[i]:round(sum(df.iloc[:,i].isnull())/len(df)*100,2) for i in range(len(df.columns))}

In [ ]:
# Imputo con la mediana las variables con marcada oblicuidad.
from sklearn.impute import SimpleImputer

X = df[['WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm']]

imputer = SimpleImputer(strategy='median',add_indicator=True)
X_imputed_median = imputer.fit_transform(X)

# Nuevo dataframe con X e indicadores de valores imputados
X_cols = ['WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm']
X_cols = X_cols+["%s_imputed" % x for x in X_cols]
X_imputed_median = pd.DataFrame(X_imputed_median,columns=X_cols)

assert(not np.any(X_imputed_median.isnull().sum()>0))
X_imputed_median.head()

In [ ]:
# Imputo con la media el resto de las variables numéricas
X_cols = ['MinTemp', 'MaxTemp', 'Rainfall', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 
          'Pressure3pm', 'Temp9am', 'Temp3pm']
X = df[X_cols]

imputer = SimpleImputer(strategy='mean',add_indicator=True)
X_imputed_mean = imputer.fit_transform(X)

# Nuevo dataframe con X e indicadores de valores imputados
X_cols = X_cols + ["%s_imputed" % x for x in X_cols]
X_imputed_mean = pd.DataFrame(X_imputed_mean,columns=X_cols)

assert(not np.any(X_imputed_mean.isnull().sum()>0))
X_imputed_mean.head()

In [ ]:
# Unificación de valores
df_imputed = pd.merge(X_imputed_mean, X_imputed_median, left_index=True, right_index=True)
df_imputed[['Location', 'Date', 'RainToday', 'RainTomorrow', 'WindGustDir','WindDir9am','WindDir3pm']] = df[['Location', 'Date', 'RainToday', 'RainTomorrow', 'WindGustDir','WindDir9am','WindDir3pm']]
df_imputed.info()

In [ ]:
# Codificación del viento

# Ángulos en radianes
direcciones_rad = {'E':0, 'ENE':np.pi*1/8, 'NE':np.pi*2/8, 'NNE':np.pi*3/8, 
                   'N':np.pi*4/8, 'NNW':np.pi*5/8, 'NW':np.pi*6/8, 'WNW':np.pi*7/8, 
                   'W':np.pi*8/8, 'WSW':np.pi*9/8, 'SW':np.pi*10/8, 'SSW':np.pi*11/8, 
                   'S':np.pi*12/8, 'SSE':np.pi*13/8, 'SE':np.pi*14/8, 'ESE':np.pi*15/8}
direcciones_rad

In [ ]:
df2 = df.dropna()
df2['angulo_WindGustDir'] = df2['WindGustDir'].map(lambda x: direcciones_rad.get(x, None))
df2['x_WindGustDir'] = df2['angulo_WindGustDir'].map(lambda x: np.cos(x))
df2['y_WindGustDir'] = df2['angulo_WindGustDir'].map(lambda y: np.cos(y))
df2['y_WindGustDir']

In [ ]:
locaciones = df['Location'].unique()
locaciones

In [ ]:
import re

def separate_names(name):
    separated_name = re.sub(r"([a-z])([A-Z])", r"\1 \2", name)
    return separated_name

# Lista de nombres
names = locaciones

# Aplicamos la función a cada nombre en la lista
separated_names = [separate_names(name) for name in names]

# Mostramos los nombres separados
for original, separated in zip(names, separated_names):
    print(f"Original: {original} - Separado: {separated}")

In [ ]:
separated_names[24] = 'Nhill'

In [ ]:
import requests

def get_city_coordinates(city_name, username):
    base_url = "http://api.geonames.org/searchJSON"
    params = {
        "q": city_name,
        "country": "AU",  
        "maxRows": 1,     
        "username": username  
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if "geonames" in data and data["geonames"]:
        city_data = data["geonames"][0]
        latitude = city_data["lat"]
        longitude = city_data["lng"]
        return latitude, longitude
    else:
        return None, None


username = "juanimunar"
cities = separated_names

coords = []
for city in cities:
    lat, lng = get_city_coordinates(city, username)
    coords.append((lat, lng))

In [ ]:
df_loc = pd.DataFrame(locaciones)
df_loc['lat-lon'] = coords
df_loc

In [ ]:
df_loc.to_csv('coordenadas.csv', index=False)